In [12]:
import numpy as np
import os
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

sns.set(font_scale=1.3)

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

In [14]:
base_model = ResNet50(
    weights='imagenet', include_top=False, input_shape=(300, 300, 3)
)



In [17]:
img_size = (300, 300)
batch_size = 32
num_classes = 2 # 가위 바위 보
epochs = 15

base_model.trainable = False

model = models.Sequential(
    [
        base_model,
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(3, activation='softmax') # 3개의 클래스 출력층
    ]
)

# 모델 요약 출력
model.summary()

model.compile(
    optimizer = Adam(learning_rate=0.001),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

train_datagen = ImageDataGenerator(
    rescale = 1./255, # 정규화
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    './data/Rock-Paper-Scissors/train/',
    target_size = img_size,
    batch_size = batch_size,
    class_mode = 'categorical'
)

validation_generator = test_datagen.flow_from_directory(
    './data/Rock-Paper-Scissors/test/',
    target_size = img_size,
    batch_size = batch_size,
    class_mode = 'categorical'
)

history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = validation_generator.samples // batch_size,
)

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 10, 10, 2048)   │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 204800)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 512)            │   104,858,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 3)              │         1,539 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 128,447,363 (489.99 MB)

 Trainable params: 104,859,651 (400.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

Found 2520 images belonging to 3 classes.
Found 372 images belonging to 3 classes.
Epoch 1/15
78/78 ━━━━━━━━━━━━━━━━━━━━ 144s 2s/step - accuracy: 0.4511 - loss: 26.4797 - val_accuracy: 0.8153 - val_loss: 0.4442
Epoch 2/15
 1/78 ━━━━━━━━━━━━━━━━━━━━ 1:57 2s/step - accuracy: 0.6250 - loss: 0.9259

c:\Users\user\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.6250 - loss: 0.9259 - val_accuracy: 0.8000 - val_loss: 0.4326
Epoch 3/15
78/78 ━━━━━━━━━━━━━━━━━━━━ 137s 2s/step - accuracy: 0.7130 - loss: 0.6447 - val_accuracy: 0.8551 - val_loss: 0.5709
Epoch 4/15
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9062 - loss: 0.4486 - val_accuracy: 0.9000 - val_loss: 0.6934
Epoch 5/15
78/78 ━━━━━━━━━━━━━━━━━━━━ 137s 2s/step - accuracy: 0.7045 - loss: 0.7710 - val_accuracy: 0.8409 - val_loss: 0.4890
Epoch 6/15
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.6250 - loss: 0.5705 - val_accuracy: 0.8000 - val_loss: 0.5744
Epoch 7/15
78/78 ━━━━━━━━━━━━━━━━━━━━ 138s 2s/step - accuracy: 0.6832 - loss: 0.6217 - val_accuracy: 0.8580 - val_loss: 0.5047
Epoch 8/15
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.6875 - loss: 0.5966 - val_accuracy: 0.8000 - val_loss: 0.5813
Epoch 9/15
78/78 ━━━━━━━━━━━━━━━━━━━━ 136s 2s/step - accuracy: 0.5810 - loss: 0.7109 - val_accuracy: 0.7557 - val_loss: 0.

KeyboardInterrupt: 